In [307]:
#!git clone https://github.com/brewing-datacup/reto-colombia-2020

In [308]:
#!pip install patool

In [309]:
#!pip install pyunpack
#from pyunpack import Archive
#Archive("/content/reto-colombia-2020/data/colombia-data.7z").extractall("/content/drive/My Drive/Reto/")

In [34]:
import pandas as pd
import numpy as np
import datetime
from sklearn import linear_model
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import metrics
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline

In [3]:
clientes_df = pd.read_csv("/content/drive/My Drive/Reto/Input1_clientes_estructura.csv", sep =';')
ventas_df = pd.read_csv("/content/drive/My Drive/Reto/Input2_clientes_venta.csv", sep =';')

In [4]:
clientes_df.head()

Cliente   Regional2   Gerencia2   SubCanal2    Categoria  Nevera
0        1  Regional 1  Gerencia_1  Subcanal_1  Categoria_1       0
1        2  Regional 1  Gerencia_1  Subcanal_1  Categoria_1       0
2        3  Regional 1  Gerencia_1  Subcanal_1  Categoria_1       0
3        4  Regional 1  Gerencia_1  Subcanal_1  Categoria_1       1
4        5  Regional 1  Gerencia_1  Subcanal_1  Categoria_2       1

In [5]:
ventas_df.head()

Año  Mes  Cliente  ...   Volumen       disc          nr
0  2019    5       10  ...  0.112229 -30.590603  900.328567
1  2019    5       10  ...  0.021734   0.000000  149.184463
2  2019    5       10  ...  0.043469   0.000000  359.625828
3  2019    5       10  ...  0.026345 -31.065261  134.748399
4  2019    5       10  ...  0.086938   0.000000  496.901005

[5 rows x 10 columns]

In [314]:
ventas_df.isnull().sum()

Año                 0
Mes                 0
Cliente             0
SegmentoPrecio2     0
Marca2              0
Cupo2               0
CapacidadEnvase2    0
Volumen             0
disc                0
nr                  0
dtype: int64

In [315]:
# clientes unicos
len(clientes_df["Cliente"].unique())

20921

In [316]:
ventas_df.shape

(1406116, 10)

In [317]:
# clientes unicos
len(ventas_df["Marca2"].unique())

39

In [318]:
ventas_df.describe()

Año           Mes  ...          disc            nr
count  1.406116e+06  1.406116e+06  ...  1.406116e+06  1.406116e+06
mean   2.019518e+03  6.865562e+00  ... -1.335855e+02  1.704834e+03
std    4.996729e-01  2.996953e+00  ...  1.491833e+03  1.047079e+04
min    2.019000e+03  1.000000e+00  ... -4.794567e+05  0.000000e+00
25%    2.019000e+03  5.000000e+00  ... -2.877037e+01  1.778085e+02
50%    2.020000e+03  7.000000e+00  ...  0.000000e+00  4.233184e+02
75%    2.020000e+03  9.000000e+00  ...  0.000000e+00  1.117963e+03
max    2.020000e+03  1.200000e+01  ...  0.000000e+00  2.302965e+06

[8 rows x 6 columns]

In [6]:
ventas_df.replace({'Marca2': r'^Marca','Cupo2': r'^Cupo','CapacidadEnvase2': r'^CapacidadEnvase'}
                  , {'Marca2': 'M', 'Cupo2':'C','CapacidadEnvase2':'CE'}, regex=True,inplace=True)

In [7]:
ventas_df['Fecha'] = pd.to_datetime(ventas_df['Año'].astype(str)+ ventas_df['Mes'].apply(lambda x: '0'+str(x) if x<=9 else str(x)) +'01')
ventas_df['Marca'] = ventas_df['Marca2']+'-'+ventas_df['Cupo2']+'-'+ventas_df['CapacidadEnvase2']

In [8]:
ventas_df.drop(columns=['Año','Mes','Marca2','Cupo2','CapacidadEnvase2'], inplace=True)

In [9]:
prueba = ventas_df

In [10]:
grouped_cliente = prueba.groupby(['Cliente','Fecha']).agg({'Volumen': ['sum'],'disc': ['sum'],'nr': ['sum'],'Marca':['nunique']}).reset_index()
grouped_cliente.columns = ['Cliente','Fecha','volumen_cliente','descuento_cliente','valor_cliente','distinct_marca_cliente']

In [11]:
grouped_cliente_marca = prueba.groupby(['Cliente','Fecha', 'Marca']).agg({'Volumen': ['sum'],'disc': ['sum'],'nr': ['sum']}).reset_index()
grouped_cliente_marca.columns = ['Cliente','Fecha','Marca','volumen_marca','descuento_marca','valor_marca']

In [13]:
prueba_2 = pd.merge(grouped_cliente, grouped_cliente_marca, left_on=['Cliente','Fecha'], right_on=['Cliente','Fecha'], how="left")

In [14]:
prueba_2['p_vol'] =prueba_2['volumen_marca']/prueba_2['volumen_cliente']
prueba_2['p_desc'] =prueba_2['descuento_marca']/prueba_2['descuento_cliente']
prueba_2['p_valor'] =prueba_2['valor_marca']/prueba_2['valor_cliente']

In [15]:
prueba_2.drop(columns=['volumen_marca','descuento_marca','valor_marca'], inplace=True)

In [16]:
prueba3= pd.pivot_table(prueba_2, values=['p_vol','p_desc','p_valor'], 
                         index=['Cliente','Fecha','volumen_cliente','descuento_cliente','valor_cliente','distinct_marca_cliente'],
                         columns=['Marca'], aggfunc=np.mean).reset_index()

In [17]:
prueba3.columns = ['_'.join(col) for col in prueba3.columns]
prueba3.rename(columns={'Cliente_': 'Cliente', 
                        'Fecha_': 'Fecha',
                        'volumen_cliente_':'volumen_cliente',
                        'descuento_cliente_':'descuento_cliente',
                        'valor_cliente_':'valor_cliente',
                        'distinct_marca_cliente_':'distinct_marca_cliente'}, inplace=True)

In [18]:
prueba3.fillna(0, inplace=True)

In [ ]:
prueba3.head()

In [19]:
ult_3_meses_train= prueba3[(prueba3['Fecha']<=pd.to_datetime('2020-07-01'))&(prueba3['Fecha']>=pd.to_datetime('2020-05-01'))]
ult_6_meses_train= prueba3[(prueba3['Fecha']<=pd.to_datetime('2020-07-01'))&(prueba3['Fecha']>=pd.to_datetime('2020-01-01'))]
ult_12_meses_train= prueba3[(prueba3['Fecha']<=pd.to_datetime('2020-07-01'))&(prueba3['Fecha']>=pd.to_datetime('2019-07-01'))]

In [20]:
ult_3_meses_test= prueba3[(prueba3['Fecha']<=pd.to_datetime('2020-08-01'))&(prueba3['Fecha']>=pd.to_datetime('2020-06-01'))]
ult_6_meses_test= prueba3[(prueba3['Fecha']<=pd.to_datetime('2020-08-01'))&(prueba3['Fecha']>=pd.to_datetime('2020-02-01'))]
ult_12_meses_test= prueba3[(prueba3['Fecha']<=pd.to_datetime('2020-08-01'))&(prueba3['Fecha']>=pd.to_datetime('2019-08-01'))]

In [21]:
columns = prueba3.columns.to_list()
columns.remove('Cliente')
columns.remove('Fecha') 

estaditicas = ['mean','median']

agg_group ={}

for i in columns :
    key=i
    agg_group[key] = estaditicas

In [22]:
ult_3_meses_train_agg = ult_3_meses_train.groupby(['Cliente']).agg(agg_group).reset_index()
ult_3_meses_train_agg.columns = ['_3_'.join(col) for col in ult_3_meses_train_agg.columns]
ult_3_meses_train_agg.rename(columns={'Cliente_3_': 'Cliente'}, inplace=True)
ult_6_meses_train_agg = ult_6_meses_train.groupby(['Cliente']).agg(agg_group).reset_index()
ult_6_meses_train_agg.columns = ['_6_'.join(col) for col in ult_6_meses_train_agg.columns]
ult_6_meses_train_agg.rename(columns={'Cliente_6_': 'Cliente'}, inplace=True)
ult_12_meses_train_agg = ult_12_meses_train.groupby(['Cliente']).agg(agg_group).reset_index()
ult_12_meses_train_agg.columns = ['_12_'.join(col) for col in ult_12_meses_train_agg.columns]
ult_12_meses_train_agg.rename(columns={'Cliente_12_': 'Cliente'}, inplace=True)

In [23]:
ult_3_meses_test_agg = ult_3_meses_test.groupby(['Cliente']).agg(agg_group).reset_index()
ult_3_meses_test_agg.columns = ['_3_'.join(col) for col in ult_3_meses_test_agg.columns]
ult_3_meses_test_agg.rename(columns={'Cliente_3_': 'Cliente'}, inplace=True)
ult_6_meses_test_agg = ult_6_meses_test.groupby(['Cliente']).agg(agg_group).reset_index()
ult_6_meses_test_agg.columns = ['_6_'.join(col) for col in ult_6_meses_test_agg.columns]
ult_6_meses_test_agg.rename(columns={'Cliente_6_': 'Cliente'}, inplace=True)
ult_12_meses_test_agg = ult_12_meses_test.groupby(['Cliente']).agg(agg_group).reset_index()
ult_12_meses_test_agg.columns = ['_12_'.join(col) for col in ult_12_meses_test_agg.columns]
ult_12_meses_test_agg.rename(columns={'Cliente_12_': 'Cliente'}, inplace=True)

In [24]:
clientes_df_dummies = pd.get_dummies(clientes_df)

In [25]:
base_train = pd.merge(clientes_df_dummies, ult_12_meses_train_agg, left_on=['Cliente'], right_on=['Cliente'],how="left")
base_train = pd.merge(base_train, ult_6_meses_train_agg, left_on=['Cliente'], right_on=['Cliente'],how="left")
base_train = pd.merge(base_train, ult_3_meses_train_agg, left_on=['Cliente'], right_on=['Cliente'],how="left")
base_test = pd.merge(clientes_df_dummies, ult_12_meses_train_agg, left_on=['Cliente'], right_on=['Cliente'],how="left")
base_test = pd.merge(base_test, ult_6_meses_train_agg, left_on=['Cliente'], right_on=['Cliente'],how="left")
base_test = pd.merge(base_test, ult_3_meses_train_agg, left_on=['Cliente'], right_on=['Cliente'],how="left")
base_test.fillna(0, inplace=True)
base_train.fillna(0, inplace=True)

In [26]:
train_filtro=base_train['Cliente'].to_list()
test_filtro=base_test['Cliente'].to_list()

In [27]:
y_train = prueba_2[prueba_2['Fecha']==pd.to_datetime('2020-08-01')]
y_train = y_train[['Cliente','Marca']]
y_train = pd.get_dummies(y_train)

columns_t = y_train.columns.to_list()
columns_t.remove('Cliente') 

estadisticas_t = ['sum']

agg_group_train ={}

for i in columns_t :
    key=i
    agg_group_train[key] = estadisticas_t
    
y_train = y_train.groupby(['Cliente']).agg(agg_group_train).reset_index()
y_train.columns = ['_'.join(col) for col in y_train.columns]
y_train.rename(columns={'Cliente_': 'Cliente'}, inplace=True)
y_train = y_train[y_train['Cliente'].isin(train_filtro)]

In [28]:
y_test = prueba_2[prueba_2['Fecha']==pd.to_datetime('2020-09-01')]
y_test = y_test[['Cliente','Marca']]
y_test = pd.get_dummies(y_test)

columns_te = y_test.columns.to_list()
columns_te.remove('Cliente') 

estadisticas_te = ['sum']

agg_group_test ={}

for i in columns_te :
    key=i
    agg_group_test[key] = estadisticas_te
    
y_test = y_test.groupby(['Cliente']).agg(agg_group_test).reset_index()
y_test.columns = ['_'.join(col) for col in y_test.columns]
y_test.rename(columns={'Cliente_': 'Cliente'}, inplace=True)
y_test = y_test[y_test['Cliente'].isin(test_filtro)]

In [29]:
train_filtro2=y_train['Cliente'].to_list()
test_filtro2=y_test['Cliente'].to_list()

In [30]:
base_train = base_train[base_train['Cliente'].isin(train_filtro2)]
base_test = base_test[base_test['Cliente'].isin(test_filtro2)]

In [31]:
y_train.set_index('Cliente' , inplace = True)
y_train.rename(columns={'Marca_M_20-C_3-CE_9_sum': 'M_20-C_3-CE_9', 
                        'Marca_M_16-C_2-CE_10_sum':'M_16-C_2-CE_10',
                        'Marca_M_9-C_3-CE_12_sum':'M_9-C_3-CE_12',
                        'Marca_M_38-C_2-CE_10_sum':'M_38-C_2-CE_10',
                        'Marca_M_39-C_2-CE_10_sum':'M_39-C_2-CE_10'
                        }, inplace=True)

In [32]:
base_train.set_index('Cliente' , inplace = True)

In [35]:

columns =["M_20-C_3-CE_9","M_16-C_2-CE_10","M_9-C_3-CE_12","M_38-C_2-CE_10","M_39-C_2-CE_10"]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(base_train)

for i in columns :
  # create the classifier
  model = RandomForestClassifier(n_estimators=100)
  y_base=y_train[i]
  X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X_scaled, y_base, test_size=0.3, random_state=8)
  # Train the model using the training sets
  model.fit(X_train, Y_train)
  #y_pred = model.predict(X_test)
  
  fpr, tpr, thresholds = metrics.roc_curve(Y_validation, model.predict_proba(X_validation)[:,1])
  auc = metrics.roc_auc_score(Y_validation, model.predict_proba(X_validation)[:,1])
  print("AUC: " + str(i)+" "+ str(auc))
  
  with open("/content/drive/My Drive/Reto/smodels{}.pickle".format(i), "wb") as f:
    pickle.dump(model, f) 


AUC: M_20-C_3-CE_9 0.7714940173769411
AUC: M_16-C_2-CE_10 0.8631150442477878
AUC: M_9-C_3-CE_12 0.8567404305964047
AUC: M_38-C_2-CE_10 0.8682733453976036
AUC: M_39-C_2-CE_10 0.8666565273362807


In [40]:
# LOAD MODEL
#y_test.set_index('Cliente' , inplace = True)
#base_test.set_index('Cliente' , inplace = True)
y_test.rename(columns={'Marca_M_20-C_3-CE_9_sum': 'M_20-C_3-CE_9', 
                        'Marca_M_16-C_2-CE_10_sum':'M_16-C_2-CE_10',
                        'Marca_M_9-C_3-CE_12_sum':'M_9-C_3-CE_12',
                        'Marca_M_38-C_2-CE_10_sum':'M_38-C_2-CE_10',
                        'Marca_M_39-C_2-CE_10_sum':'M_39-C_2-CE_10'
                        }, inplace=True)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(base_test)


for i in columns: 
   with open("/content/drive/My Drive/Reto/smodels{}.pickle".format(i), "rb") as f:
     model = pickle.load(f)
     
     predicted= model.predict(X_scaled)
     fpr, tpr, thresholds = metrics.roc_curve(y_test[i], model.predict_proba(X_scaled)[:,1])
     auc = roc_auc_score(y_test[i], model.predict_proba(X_scaled)[:,1])
     print("AUC: " + str(i)+" "+ str(auc))
    

AUC: M_20-C_3-CE_9 0.7809587824395163
AUC: M_16-C_2-CE_10 0.8026123701555311
AUC: M_9-C_3-CE_12 0.8363206302029949
AUC: M_38-C_2-CE_10 0.8252011738656142
AUC: M_39-C_2-CE_10 0.8152034294224966


In [41]:
base_pred= pd.read_csv("/content/drive/My Drive/Reto/Input3_clientes_test.csv", sep=";")

In [43]:
base_filtro=base_pred['Cliente'].to_list()
prueba4 = prueba3[prueba3['Cliente'].isin(base_filtro)]
clientes_df_dummies_2= clientes_df_dummies[clientes_df_dummies["Cliente"].isin(base_filtro)]

In [44]:
ult_3_meses_base= prueba4[(prueba4['Fecha']<=pd.to_datetime('2020-09-01'))&(prueba3['Fecha']>=pd.to_datetime('2020-07-01'))]
ult_6_meses_base= prueba4[(prueba4['Fecha']<=pd.to_datetime('2020-09-01'))&(prueba3['Fecha']>=pd.to_datetime('2020-03-01'))]
ult_12_meses_base= prueba4[(prueba4['Fecha']<=pd.to_datetime('2020-09-01'))&(prueba3['Fecha']>=pd.to_datetime('2019-09-01'))]

ult_3_meses_base_agg = ult_3_meses_base.groupby(['Cliente']).agg(agg_group).reset_index()
ult_3_meses_base_agg.columns = ['_3_'.join(col) for col in ult_3_meses_base_agg.columns]
ult_3_meses_base_agg.rename(columns={'Cliente_3_': 'Cliente'}, inplace=True)
ult_6_meses_base_agg = ult_6_meses_base.groupby(['Cliente']).agg(agg_group).reset_index()
ult_6_meses_base_agg.columns = ['_6_'.join(col) for col in ult_6_meses_base_agg.columns]
ult_6_meses_base_agg.rename(columns={'Cliente_6_': 'Cliente'}, inplace=True)
ult_12_meses_base_agg = ult_12_meses_base.groupby(['Cliente']).agg(agg_group).reset_index()
ult_12_meses_base_agg.columns = ['_12_'.join(col) for col in ult_12_meses_base_agg.columns]
ult_12_meses_base_agg.rename(columns={'Cliente_12_': 'Cliente'}, inplace=True)

base_fin = pd.merge(clientes_df_dummies_2,ult_12_meses_base_agg,left_on=['Cliente'], right_on=['Cliente'], how="left")
base_fin= pd.merge(base_fin, ult_6_meses_base_agg, left_on=['Cliente'], right_on=['Cliente'],how="left")
base_fin= pd.merge(base_fin, ult_3_meses_base_agg, left_on=['Cliente'], right_on=['Cliente'],how="left")
base_fin.set_index('Cliente' , inplace = True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


In [45]:
base_fin.fillna(0, inplace=True)

In [47]:
scaler = StandardScaler()
X_scaled1 = scaler.fit_transform(base_fin)

for i in columns: 
   with open("/content/drive/My Drive/Reto/smodels{}.pickle".format(i), "rb") as f:
     model = pickle.load(f)
     base_fin[i]=np.round(model.predict_proba(X_scaled1)[:,1],3)#df['column'].round(1)
base_predic=base_fin[columns]

In [48]:
#base_predic.reset_index(inplace=True)
base_predic.columns = ['Marca1',	'Marca2',	'Marca3',	'Marca_Inno1',	'Marca_Inno2']


In [ ]:
lista1 = prueba4["Cliente"].astype(str).to_list()
lista2 = base_predic["Cliente"].astype(str).to_list()
different = []
 
for item in lista1:
    x = True
    for item2 in lista2:
          if item2 in item:
              x = False
              break
    if x:
        different.append(item)
 
print(different)

In [50]:
base_predic.to_csv("/content/drive/My Drive/Reto/output.csv")

In [49]:
base_predic

Marca1  Marca2  Marca3  Marca_Inno1  Marca_Inno2
Cliente                                                  
1         0.270    0.00   0.100         0.51        0.520
2         0.340    0.00   0.080         0.10        0.150
3         0.280    0.00   0.100         0.52        0.440
4         0.560    0.01   0.160         0.41        0.380
5         0.480    0.00   0.350         0.24        0.170
...         ...     ...     ...          ...          ...
20411     0.180    0.00   0.090         0.90        0.910
20414     0.196    0.00   0.164         0.82        0.710
20434     0.211    0.00   0.161         0.81        0.743
20462     0.200    0.00   0.200         0.85        0.820
20569     0.150    0.00   0.179         0.73        0.680

[9302 rows x 5 columns]